# **필요한 라이브러리 불러오기**

In [104]:
import pandas as pd
import time
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# **기본 데이터 수집**

## **2019 - 2021년 기관별 기본정보 및 손익계산서**
* 한국보건산업진흥원의 의료기관 회계공시 데이터를 웹 스크랩핑함

In [ ]:
# 전체 데이터를 담을 빈 리스트 생성
data = []

for item in [[2021, 29, 285], [2020, 29, 282], [2019, 28, 275]]:
    
    # 연도별 마지막 페이지 및 마지막 순번 지정
    year = item[0]
    last_page = item[1]
    last_num = item[2]

    # 마지막 순번에서 멈출 조건 지정
    break_condition = False

    for page in range(1, last_page + 1):

        # 페이지 순서대로 열기
        url = 'https://haspa.khidi.or.kr/total-public-inq?p={}&y={}&hn=&bn=&en=&a1=&a2='.format(page, year)
        service = Service('./driver/chromedriver.exe')
        driver = webdriver.Chrome(service = service)
        actions = ActionChains(driver)
        driver.get(url)

        for i in range(2, 12):

            # 개별 기관의 정보를 담을 리스트 생성 (연도 정보만 담음)
            tmp = [year]
            
            # 기본정보 가져오기
            number = int(driver.find_element(By.XPATH, '//*[@id="content_data"]/div[2]/div[' + str(i) + ']/div[1]').text)
            time.sleep(0.2)
            name = driver.find_element(By.XPATH, '//*[@id="content_data"]/div[2]/div[' + str(i) + ']/div[3]/span').text
            time.sleep(0.2)
            num_of_beds = int(driver.find_element(By.XPATH, '//*[@id="content_data"]/div[2]/div[' + str(i) + ']/div[4]').text)
            time.sleep(0.2)
            tertiary = driver.find_element(By.XPATH, '//*[@id="content_data"]/div[2]/div[' + str(i) + ']/div[6]').text
            time.sleep(0.2)
            incor_type = driver.find_element(By.XPATH, '//*[@id="content_data"]/div[2]/div[' + str(i) + ']/div[7]').text
            time.sleep(0.2)

            # 기본정보 가져와서 tmp에 담기
            for info in [number, name, num_of_beds, tertiary, incor_type]:
                tmp.append(info)

            # 손익계산서 창 열기 버튼 XPATH 정의
            inc_st = '//*[@id="content_data"]/div[2]/div['+ str(i) +']/div[10]'
            
            # 손익계산서 창 열기
            inc_st_button = driver.find_element(By.XPATH, inc_st)
            inc_st_button.click()
            time.sleep(3)

            # 기관별 주소 정보 가져와서 tmp에 담기
            tmp.append(driver.find_element(By.XPATH, '//*[@id="is-addr"]').text)

            # 손익계산서 숫자 가져와서 tmp에 담기
            table = driver.find_element(By.XPATH, '//*[@id="isTable"]')
            tbody = table.find_element(By.TAG_NAME, 'tbody')
            rows = tbody.find_elements(By.TAG_NAME, 'tr')
                    
            for value in rows:
                actions.move_to_element(value).perform()
                body = value.find_elements(By.TAG_NAME, 'td')[1]
                tmp.append(body.text.replace(',', ''))
                time.sleep(0.2)
            
            # 숫자 모두 가져왔으면 손익계산서 창 닫기
            out_button = driver.find_element(By.XPATH, '//*[@id="is-modal"]/div/div/div[1]/div[1]/button')
            actions.move_to_element(out_button).perform()
            out_button.send_keys(Keys.ENTER)

            # tmp에 담은 기관별 데이터 전체 데이터 리스트에 담기
            data.append(tmp)

            # 마지막 순번이 되면 해당 연도 Loop 종료
            if number == last_num:
                break_condition = True
                break
        
        # 웹드라이버 종료
        driver.quit()
        time.sleep(2)

        # 마지막 순번이 되면 해당 연도 Loop 종료
        if break_condition == True:
            break

before_process = pd.DataFrame(data)
before_process.to_excel('./before_process.xlsx')

## **손익계산서 계정명**

In [ ]:
# 컬럼명을 저장할 빈 리스트 저장하기
column_names = []

# 컬럼명을 불러올 샘플 페이지 열기
url = 'https://haspa.khidi.or.kr/total-public-inq?p={}&y={}&hn=&bn=&en=&a1=&a2='.format(1, 2021)
service = Service('./driver/chromedriver.exe')
driver = webdriver.Chrome(service = service)
actions = ActionChains(driver)
driver.get(url)

# 손익계산서 창 열기 버튼 XPATH 정의
inc_st = '//*[@id="content_data"]/div[2]/div['+ str(2) +']/div[10]'

# 손익계산서 창 열기
inc_st_button = driver.find_element(By.XPATH, inc_st)
inc_st_button.click()
time.sleep(3)

# 손익계산서 계정명 가져오기
table = driver.find_element(By.XPATH, '//*[@id="isTable"]')
tbody = table.find_element(By.TAG_NAME, 'tbody')
rows = tbody.find_elements(By.TAG_NAME, 'tr')

for value in rows:
    actions.move_to_element(value).perform()
    body = value.find_elements(By.TAG_NAME, 'td')[0]
    column_names.append(body.text.replace(' ', ''))
    time.sleep(0.2)

# 웹드라이버 종료
driver.quit()
time.sleep(2)

# 컬럼명 리스트 지정
column_names = ['연도', '순번', '기관명', '병상수', '종별', '설립형태', '주소'] + column_names

# 저장된 컬럼명 데이터 프레임에 반영
before_process.columns = column_names

# 필요하지 않은 '순번' 컬럼은 삭제
tmp = before_process.columns.tolist()
tmp.remove('순번')
before_process = before_process[tmp]

# **전처리**

## **도로명 주소만 남기기**
* 가져온 주소는 도로명 주소 이외의 정보도 담겨 있음
* 주소 데이터로 각 병원의 경도 및 위도를 확보하기 위해 도로명 주소만 남기고자 함

In [ ]:
for idx in range(before_process.shape[0]):
    before_process.loc[idx, '주소'] = before_process.loc[idx, '주소'].split(')')[0].rstrip()

## **결측치가 존재하는지 확인하기**

In [ ]:
null_condition = 0

for column in before_process.columns.tolist():
    null_condition += sum(before_process.isnull()[column])

if null_condition == 0:
    print('결측치가 존재하지 않습니다.')
else:
    print('결측치가 존재합니다.')

# 전처리된 데이터 중간 저장
before_process.to_excel('./after_process.xlsx')

결측치가 존재하지 않습니다.


## **세목을 제외한 데이터만 남기기**
* 의료기관 회계기준 규칙에 규정된 손익계산서의 계정과목은 100개가 넘음
* 그 중 세목을 제외한 상위 계정과목 50여개만을 남겨 분석에 활용하고자 함
* 상위 계정과 하위 계정 간의 관계를 바로 파악할 수 있도록 컬럼명을 수정함
    * 수정 전 : Ⅳ.의료외수익 - 1.의료부대수익
    * 수정 후 : Ⅳ.의료외수익 - Ⅳ-1.의료부대수익

In [ ]:
# 1차 전처리된 데이터 읽어오기
after_process = pd.read_excel('./after_process.xlsx', index_col = 0)

# 세목을 제외한 상위 계정과목을 데이터 프레임에 남기기
target_columns = ['연도', '기관명', '병상수', '종별', '설립형태', '주소',
                  'Ⅰ.의료수익', '1.입원수익', '2.외래수익', '3.기타의료수익',
                  'Ⅱ.의료비용', '1.인건비', '2.재료비', '3.관리운영비', '4.기타의료비용',
                  'Ⅲ.의료이익(손실)',
                  'Ⅳ.의료외수익', '1.의료부대수익', '2.이자수익', '3.배당금수익', '4.임대료수익', 
                                  '5.단기매매증권처분이익', '6.단기매매증권평가이익', '7.연구수익', 
                                  '8.외환차익', '9.외화환산이익', '10.투자자산처분이익', '11.유형자산처분이익', 
                                  '12.대손충당금환입', '13.기부금수익', '14.잡이익', '15.자산수증이익', 
                                  '16.채무면제이익', '17.보험차익', '18.기타의료외수익',
                  'Ⅴ.의료외비용', '1.의료부대비용', '2.이자비용', '3.기타대손상각비', '4.기부금', 
                                  '5.단기매매증권처분손실', '6.단기매매증권평가손실', '7.연구비용',
                                  '8.외환차손', '9.외화환산손실', '10.투자자산처분손실', '11.유형자산처분손실',
                                  '12.고유목적사업비', '13.잡손실', '14.재해손실', '15.기타의료외비용',
                  'Ⅵ.법인세차감전순이익(순손실)', 'Ⅶ.법인세비용(소득세)', 'Ⅷ.고유목적사업준비금설정전당기순이익',
                  'Ⅸ.고유목적사업준비금전입액¹', 'Ⅹ.고유목적사업준비금환입액¹', '당기순이익(순손실)']
target_data = after_process[target_columns]

# 컬럼명 수정하여 상위 계정과 하위 계정간의 관계를 알 수 있도록 함 
target_columns = ['연도', '기관명', '병상수', '종별', '설립형태', '주소',
                  'Ⅰ.의료수익', 'Ⅰ-1.입원수익', 'Ⅰ-2.외래수익', 'Ⅰ-3.기타의료수익',
                  'Ⅱ.의료비용', 'Ⅱ-1.인건비', 'Ⅱ-2.재료비', 'Ⅱ-3.관리운영비', 'Ⅱ-4.기타의료비용',
                  'Ⅲ.의료이익(손실)',
                  'Ⅳ.의료외수익', 'Ⅳ-1.의료부대수익', 'Ⅳ-2.이자수익', 'Ⅳ-3.배당금수익', 'Ⅳ-4.임대료수익', 
                                  'Ⅳ-5.단기매매증권처분이익', 'Ⅳ-6.단기매매증권평가이익', 'Ⅳ-7.연구수익', 
                                  'Ⅳ-8.외환차익', 'Ⅳ-9.외화환산이익', 'Ⅳ-10.투자자산처분이익', 'Ⅳ-11.유형자산처분이익', 
                                  'Ⅳ-12.대손충당금환입', 'Ⅳ-13.기부금수익', 'Ⅳ-14.잡이익', 'Ⅳ-15.자산수증이익', 
                                  'Ⅳ-16.채무면제이익', 'Ⅳ-17.보험차익', 'Ⅳ-18.기타의료외수익',
                  'Ⅴ.의료외비용', 'Ⅴ-1.의료부대비용', 'Ⅴ-2.이자비용', 'Ⅴ-3.기타대손상각비', 'Ⅴ-4.기부금', 
                                  'Ⅴ-5.단기매매증권처분손실', 'Ⅴ-6.단기매매증권평가손실', 'Ⅴ-7.연구비용',
                                  'Ⅴ-8.외환차손', 'Ⅴ-9.외화환산손실', 'Ⅴ-10.투자자산처분손실', 'Ⅴ-11.유형자산처분손실',
                                  'Ⅴ-12.고유목적사업비', 'Ⅴ-13.잡손실', 'Ⅴ-14.재해손실', 'Ⅴ-15.기타의료외비용',
                  'Ⅵ.법인세차감전순이익(순손실)', 'Ⅶ.법인세비용(소득세)', 'Ⅷ.고유목적사업준비금설정전당기순이익',
                  'Ⅸ.고유목적사업준비금전입액¹', 'Ⅹ.고유목적사업준비금환입액¹', '당기순이익(순손실)']

target_data.columns = target_columns

## **동일한 주소의 기관명이 연도별로 다른 경우 최근 기준으로 통일**

In [185]:
error_address = []
latest_name = []

address_list = target_data['주소'].tolist()

for address in list(set(address_list)):
    length = len(list(set(target_data[target_data['주소'] == address]['기관명'])))
    if length != 1:
        print('{}의 기관명을 확인해주시기 바랍니다'.format(address))
        print('기관명 : ',target_data[target_data['주소'] == address]['기관명'].tolist(), '\n')
        error_address.append(address)
        latest_name.append(target_data[target_data['주소'] == address]['기관명'].tolist()[0])

correct_dict = {error : latest for error, latest in zip(error_address, latest_name)}

경기도 시흥시 군자천로 381의 기관명을 확인해주시기 바랍니다
기관명 :  ['의료법인 남촌의료재단 시화병원', '의료법인 남촌의료재단 시흥시화병원'] 

경상남도 양산시 신기로 28의 기관명을 확인해주시기 바랍니다
기관명 :  ['베데스다복음병원', '베데스다병원', '베데스다병원'] 

충청북도 진천군 진천읍 중앙북로 24의 기관명을 확인해주시기 바랍니다
기관명 :  ['의료법인 건명의료재단 중앙제일병원', '의료법인 건명의료재단 진천성모병원', '의료법인 건명의료재단 진천성모병원'] 

경상남도 김해시 장유로 167-13의 기관명을 확인해주시기 바랍니다
기관명 :  ['의료법인갑을의료재단 갑을장유병원', '의료법인갑을의료재단갑을장유병원', '의료법인갑을의료재단갑을장유병원'] 

경상남도 창원시 성산구 삼정자로 11의 기관명을 확인해주시기 바랍니다
기관명 :  ['창원경상국립대학교병원', '창원경상대학교병원', '창원경상대학교병원'] 

경상북도 경산시 경안로 11의 기관명을 확인해주시기 바랍니다
기관명 :  ['의료법인 근원의료재단 경산중앙병원', '의)근원의료재단경산중앙병원', '의)근원의료재단경산중앙병원'] 

세종특별자치시 한누리대로 161의 기관명을 확인해주시기 바랍니다
기관명 :  ['의료법인 영제 의료재단 엔케이세종병원', '엔케이(NK)세종병원'] 

부산광역시 부산진구 가야대로 721의 기관명을 확인해주시기 바랍니다
기관명 :  ['의료법인 온그룹의료재단 온종합병원', '의료법인정선의료재단 온종합병원', '의료법인정선의료재단 온종합병원'] 

경상남도 진주시 강남로 79의 기관명을 확인해주시기 바랍니다
기관명 :  ['경상국립대학교병원', '경상대학교병원', '경상대학교병원'] 

경기도 군포시 산본로 321의 기관명을 확인해주시기 바랍니다
기관명 :  ['원광대학교 산본병원', '원광대학교의과대학산본병원', '원광대학교의과대학산본병원'] 

인천광역시 서구 칠천왕로33번길 17의 기관명을 확인해주시기 바랍니다
기관명 :  ['(의)성세의료

In [190]:
for idx in tqdm(range(target_data.shape[0])):
    if target_data.loc[idx, '주소'] in error_address:
        target_data.loc[idx, '기관명'] = correct_dict[target_data.loc[idx, '주소']]

100%|██████████| 842/842 [00:00<00:00, 31024.16it/s]


In [193]:
target_data[target_data['주소'] == '경기도 시흥시 군자천로 381'][['기관명', '주소']]

,기관명,주소
128,의료법인 남촌의료재단 시화병원,경기도 시흥시 군자천로 381
412,의료법인 남촌의료재단 시화병원,경기도 시흥시 군자천로 381


## **동일한 기관의 주소가 연도별로 다른 경우 최근 기준으로 통일**

In [194]:
error_name = []
latest_address = []

name_list = target_data['기관명'].tolist()

for name in list(set(name_list)):
    length = len(list(set(target_data[target_data['기관명'] == name]['주소'])))
    if length != 1:
        print('{}의 주소 정보를 확인해주시기 바랍니다'.format(name))
        print('주소 : ', target_data[target_data['기관명'] == name]['주소'].tolist(), '\n')
        error_name.append(name)
        latest_address.append(target_data[target_data['기관명'] == name]['주소'].tolist()[0])

correct_dict = {error : latest for error, latest in zip(error_name, latest_address)}

의료법인 인천사랑병원의 주소 정보를 확인해주시기 바랍니다
주소 :  ['인천광역시 미추홀구 미추홀대로 726', '인천광역시 남구 미추홀대로 726', '인천광역시 남구 미추홀대로 726'] 

서울산보람병원의 주소 정보를 확인해주시기 바랍니다
주소 :  ['울산광역시 울주군 삼남읍 중평로 53', '울산광역시 울주군 삼남면 중평로 53', '울산광역시 울주군 삼남면 중평로 53'] 

연세대학교 의과대학 용인세브란스병원의 주소 정보를 확인해주시기 바랍니다
주소 :  ['경기도 용인시 기흥구 동백죽전대로 363', '경기도 용인시 기흥구 동백죽전대로 363', '경기도 용인시 처인구 금학로 225'] 



In [199]:
for idx in tqdm(range(target_data.shape[0])):
    if target_data.loc[idx, '기관명'] in error_name:
        target_data.loc[idx, '주소'] = correct_dict[target_data.loc[idx, '기관명']]

100%|██████████| 842/842 [00:00<00:00, 52457.61it/s]


In [201]:
target_data[target_data['기관명'] == '연세대학교 의과대학 용인세브란스병원'][['기관명', '주소']]

,기관명,주소
112,연세대학교 의과대학 용인세브란스병원,경기도 용인시 기흥구 동백죽전대로 363
395,연세대학교 의과대학 용인세브란스병원,경기도 용인시 기흥구 동백죽전대로 363
673,연세대학교 의과대학 용인세브란스병원,경기도 용인시 기흥구 동백죽전대로 363


In [203]:
target_data.to_excel('./target_data.xlsx')

## **그 외 데이터 수정**
* 경우 ① : 2021년에 처음 종합병원으로 지정된 경우 - 수정 X
* 경우 ② : 2021 - 2019년 동안 주소와 기관명이 모두 바뀐 경우 - Excel로 직접 데이터 확인 후 개별적으로 변경

In [202]:
name_list = target_data['기관명'].tolist()
error_name = []

for name in list(set(name_list)):
    if name_list.count(name) == 1:
        print('{}의 기관명을 다시 한번 확인해주시기 바랍니다'.format(name))
        error_name.append(name)

창원한마음병원의 기관명을 다시 한번 확인해주시기 바랍니다
의료법인 명인의료재단 화홍병원의 기관명을 다시 한번 확인해주시기 바랍니다
세종충남대학교병원의 기관명을 다시 한번 확인해주시기 바랍니다
빛고을전남대학교병원의 기관명을 다시 한번 확인해주시기 바랍니다
의료법인 남촌의료재단 시흥시화병원의 기관명을 다시 한번 확인해주시기 바랍니다
학교법인 을지학원 의정부을지대학교병원의 기관명을 다시 한번 확인해주시기 바랍니다
의료법인 행도의료재단 해동병원의 기관명을 다시 한번 확인해주시기 바랍니다


In [204]:
# 창원한마음병원과 의료법인 남촌의료재단 시흥시화병원을 제외하면 
# 모두 경우 ②에 해당하여 수정할 필요가 없음

# 창원한마음병원 수정
target_data.replace({'기관명' : {'의료법인 한마음국제의료재단 한마음창원병원' : '창원한마음병원'},
                     '주소' : {'경상남도 창원시 성산구 원이대로682번길 21' : '경상남도 창원시 의창구 용동로57번길 8'}},
                     inplace = True)

# 의료법인 남촌의료재단 시흥시화병원
target_data.replace({'기관명' : {'의료법인 남촌의료재단 시흥시화병원' : '의료법인 남촌의료재단 시화병원'},
                     '주소' : {'경기도 시흥시 옥구천서로 337' : '경기도 시흥시 군자천로 381'}},
                     inplace = True)

C:\Users\nuno3\AppData\Local\Temp\ipykernel_14908\2212188480.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_data.replace({'기관명' : {'의료법인 한마음국제의료재단 한마음창원병원' : '창원한마음병원'},
C:\Users\nuno3\AppData\Local\Temp\ipykernel_14908\2212188480.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_data.replace({'기관명' : {'의료법인 남촌의료재단 시흥시화병원' : '의료법인 남촌의료재단 시화병원'},


In [205]:
target_data[target_data['기관명'] == '의료법인 남촌의료재단 시화병원'][['기관명', '주소']]

,기관명,주소
128,의료법인 남촌의료재단 시화병원,경기도 시흥시 군자천로 381
412,의료법인 남촌의료재단 시화병원,경기도 시흥시 군자천로 381
690,의료법인 남촌의료재단 시화병원,경기도 시흥시 군자천로 381


In [206]:
target_data.to_excel('./target_data.xlsx')

# **추가 데이터 수집 (기관별 위도 및 경도)**

## **각 병원의 경도 및 위도 데이터 확보하기**
* 국토교통부 공간정보산업진흥원의 지오코딩 API를 활용하여 각 병원의 위도 및 경도 데이터 확보
* API를 활용하는 과정에서 주소 데이터가 잘못된 경우(API가 위도 및 경도를 모두 0으로 반환)가 발견되어 도로명 주소를 수정함
    * 예제
        * 잘못된 주소 : 인천광역시 남구 미추홀대로 726
        * 실제 주소 : 인천광역시 미추홀구 미추홀대로 726

In [212]:
url = 'http://api.vworld.kr/req/address?'
params = 'service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type='
road_type = 'ROAD' #도로명 주소
address = '&address='
keys = '&key='
primary_key = 'your_key' # 공간정보산업진흥원 API키를 발급받아 입력

def request_geo(road):
    page = requests.get(url+params+road_type+address+road+keys+primary_key)
    json_data = page.json()
    if json_data['response']['status'] == 'OK':
        x = json_data['response']['result']['point']['x']
        y = json_data['response']['result']['point']['y']
        return x, y
    else:
        x = 0
        y = 0
        return x, y

error_address = []

for idx in tqdm(range(target_data.shape[0])):
    x, y  = request_geo(target_data.loc[idx, '주소'])
    if (x == 0) and (y == 0):
        error_address.append(target_data.loc[idx, '주소'])

error_address = list(set(error_address))
error_address

100%|██████████| 842/842 [00:55<00:00, 15.24it/s]


['서울특별시 성북구 인촌로 73', '인천광역시 남구 독배로 503', '울산광역시 동구 방어진순환도로 877']

In [214]:
address_fix = {'울산광역시 동구 방어진순환도로 877' : '울산광역시 동구 대학병원로 25',
               '서울특별시 성북구 인촌로 73' : '서울특별시 성북구 고려대로 73',
               '인천광역시 남구 독배로 503' : '인천광역시 미추홀구 독배로 503'}

for idx in range(target_data.shape[0]):
    if target_data.loc[idx, '주소'] in error_address:
        target_data.loc[idx, '주소'] = address_fix[target_data.loc[idx, '주소']]

In [215]:
target_data[target_data['주소'] == '울산광역시 동구 대학병원로 25'][['연도', '기관명', '주소']]

,연도,기관명,주소
258,2021,학교법인 울산공업학원 울산대학교병원,울산광역시 동구 대학병원로 25
541,2020,학교법인 울산공업학원 울산대학교병원,울산광역시 동구 대학병원로 25
816,2019,학교법인 울산공업학원 울산대학교병원,울산광역시 동구 대학병원로 25


In [218]:
longitude = []
latitude = []
error_condition = 0

for idx in tqdm(range(target_data.shape[0])):
    x, y  = request_geo(target_data.loc[idx, '주소'])
    longitude.append(float(x))
    latitude.append(float(y))
    if (x == 0) and (y == 0):
        print('오류 발생 : {}'.format(target_data.loc[idx, '주소']))
        error_condition += 1

if error_condition == 0:
    print('정상적으로 위도 및 경도값이 반환되었습니다')
else:
    print('위치 정보를 확인해주시기 바랍니다.')

100%|██████████| 842/842 [00:49<00:00, 16.85it/s]

정상적으로 위도 및 경도값이 반환되었습니다


In [220]:
target_data['위도'] = latitude
target_data['경도'] = longitude

C:\Users\nuno3\AppData\Local\Temp\ipykernel_14908\3248646445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_data['위도'] = latitude
C:\Users\nuno3\AppData\Local\Temp\ipykernel_14908\3248646445.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_data['경도'] = longitude


In [222]:
target_data.head()

,연도,기관명,병상수,종별,설립형태,주소,Ⅰ.의료수익,Ⅰ-1.입원수익,Ⅰ-2.외래수익,Ⅰ-3.기타의료수익,...,Ⅴ-14.재해손실,Ⅴ-15.기타의료외비용,Ⅵ.법인세차감전순이익(순손실),Ⅶ.법인세비용(소득세),Ⅷ.고유목적사업준비금설정전당기순이익,Ⅸ.고유목적사업준비금전입액¹,Ⅹ.고유목적사업준비금환입액¹,당기순이익(순손실),위도,경도
0,2021,(의)나사렛의료재단 나사렛국제병원,301,종합병원,의료법인,인천광역시 연수구 먼우금로 98,79081905450,44082709774,24175326743,10823868933,...,0,99189330,14058376419,2504797573,11553578846,11000000000,2117258751,2670837597,37.407889,126.670475
1,2021,(의)성세의료재단 뉴성민병원,250,종합병원,의료법인,인천광역시 서구 칠천왕로33번길 17,36309457028,25767334917,7967048011,2575074100,...,0,46162000,-227529042,-48283340,-179245702,0,0,-179245702,37.509161,126.669750
2,2021,(의)영문의료재단 다보스병원,183,종합병원,의료법인,경기도 용인시 처인구 백옥대로1082번길 18,53456662441,32946496329,16517728346,3992437766,...,0,0,6120922459,368436435,5752486024,3610000000,475661687,2618147711,37.231536,127.211423
3,2021,(재)미리내천주성삼성직수도회천주성삼병원,264,종합병원,재단법인,대구광역시 수성구 달구벌대로 3190,45828954815,30244739757,12115090160,3469124898,...,0,1050000,3821948169,210114820,3611833349,0,0,3611833349,35.840224,128.705721
4,2021,가톨릭관동대학교 국제성모병원,618,종합병원,학교법인,인천광역시 서구 심곡로100번길 25,185298525567,116069463139,59426317322,9802745106,...,0,485032103,7826789783,821455025,7005334758,0,0,7005334758,37.542331,126.683608


# **데이터 저장**

In [223]:
target_data.to_excel('./final_data.xlsx')